# Data Transformations

Vous n'obtiendrez pas toujours les données dans un format pratique, vous devrez souvent traiter des données non numériques, telles que des noms de clients, des codes postaux, des noms de pays, etc.

Une grande partie du travail avec les données consiste à utiliser vos propres connaissances de domaine pour créer une intuition sur la manière de traiter les données. Parfois, la meilleure solution consiste à supprimer les données, parfois, l’ingénierie des caractéristiques "feature-engineering" est un bon choix ou vous pouvez essayer de transformer les données en quelque chose que les algorithmes d'apprentissage automatique vont comprendre.

Spark a plusieurs méthodes intégrées pour gérer ces transformations, vérifiez-les toutes ici:http://spark.apache.org/docs/latest/ml-features.html



In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('data').getOrCreate()

In [3]:
df = spark.read.csv('fake_customers.csv',inferSchema=True,header=True)

In [4]:
df.show()

+-------+----------+-----+
|   Name|     Phone|Group|
+-------+----------+-----+
|   John|4085552424|    A|
|   Mike|3105552738|    B|
| Cassie|4085552424|    B|
|  Laura|3105552438|    B|
|  Sarah|4085551234|    A|
|  David|3105557463|    C|
|   Zach|4085553987|    C|
|  Kiera|3105552938|    A|
|  Alexa|4085559467|    C|
|Karissa|3105553475|    A|
+-------+----------+-----+



## Les caractéristiques de données 

### StringIndexer

Nous devons souvent convertir les informations en chaîne de caractères en informations numériques en tant que caractéristique catégorique. Cela se fait facilement avec la méthode StringIndexer:

In [5]:
from pyspark.ml.feature import StringIndexer

df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["user_id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(df).transform(df)
indexed.show()

+-------+--------+-------------+
|user_id|category|categoryIndex|
+-------+--------+-------------+
|      0|       a|          0.0|
|      1|       b|          2.0|
|      2|       c|          1.0|
|      3|       a|          0.0|
|      4|       a|          0.0|
|      5|       c|          1.0|
+-------+--------+-------------+



### VectorIndexer

VectorAssembler est un transformateur qui combine une liste donnée de colonnes en une seule colonne de vecteur. Il est utile pour combiner des entités brutes et des entités générées par différents transformateurs d’entités en un seul vecteur d’entités, afin de former des modèles ML tels que la régression logistique et les arbres de décision. VectorAssembler accepte les types de colonne d'entrée suivants: tous les types numériques, le type booléen et le type de vecteur. Dans chaque ligne, les valeurs des colonnes d'entrée seront concaténées en un vecteur dans l'ordre spécifié.

Supposons que nous ayons un DataFrame avec les colonnes id, hour, mobile, userFeatures et clicked:

     id | hour | mobile | userFeatures     | clicked
    ----|------|--------|------------------|---------
     0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0
     

userFeatures est une colonne de vecteur contenant trois fonctionnalités utilisateur. Nous voulons combiner les fonctions heure, mobile et userFeatures en un seul vecteur appelé fonctionnalités et l’utiliser pour prédire les clics ou non. Si nous définissons les colonnes d'entrée de VectorAssembler sur hour, mobile et userFeatures et la colonne de sortie sur les entités, après transformation, nous devons obtenir le DataFrame suivant:

     id | hour | mobile | userFeatures     | clicked | features
    ----|------|--------|------------------|---------|-----------------------------
     0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0     | [18.0, 1.0, 0.0, 10.0, 0.5]

In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

dataset = spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
    ["id", "hour", "mobile", "userFeatures", "clicked"])
dataset.show()

+---+----+------+--------------+-------+
| id|hour|mobile|  userFeatures|clicked|
+---+----+------+--------------+-------+
|  0|  18|   1.0|[0.0,10.0,0.5]|    1.0|
+---+----+------+--------------+-------+



In [7]:
assembler = VectorAssembler(
    inputCols=["hour", "mobile", "userFeatures"],
    outputCol="features")

output = assembler.transform(dataset)
print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
output.select("features", "clicked").show()

Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'
+--------------------+-------+
|            features|clicked|
+--------------------+-------+
|[18.0,1.0,0.0,10....|    1.0|
+--------------------+-------+




De nombreuses autres transformations de données sont disponibles, nous les couvrirons dès que nous en aurons un besoin.
